<a href="https://colab.research.google.com/github/arthurezende/rankingFaculdadesMec/blob/main/Projeto_dados_MEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependências
!pip install pandas openpyxl
import pandas as pd

In [2]:
# Conectar com Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Função para carregar DataFrame e tratar erros, lendo apenas colunas específicas
def load_dataframe(filepath, filetype, columns, **kwargs):
    try:
        if filetype == 'csv':
            return pd.read_csv(filepath, usecols=columns, **kwargs)
        elif filetype == 'excel':
            return pd.read_excel(filepath, usecols=columns, **kwargs)
    except FileNotFoundError:
        print(f"Arquivo {filepath} não encontrado.")
        return None
    except Exception as e:
        print(f"Erro ao carregar o arquivo {filepath}: {e}")
        return None

# Selecionar colunas relevantes
colunas_ies = ['NU_ANO_CENSO', 'SG_UF_IES', 'NO_MUNICIPIO_IES', 'TP_ORGANIZACAO_ACADEMICA', 'TP_REDE', 'CO_IES', 'NO_IES']
colunas_cursos = ['NU_ANO_CENSO', 'SG_UF', 'NO_MUNICIPIO', 'TP_REDE', 'CO_IES', 'NO_CURSO', 'CO_CURSO', 'NO_CINE_ROTULO', 'NO_CINE_AREA_ESPECIFICA', 'TP_GRAU_ACADEMICO', 'IN_GRATUITO', 'TP_MODALIDADE_ENSINO', 'QT_VG_TOTAL', 'QT_INSCRITO_TOTAL']
colunas_emec_ies = ['Código IES', 'CI', 'Ano CI', 'CI-EaD', 'Ano CI-EaD', 'IGC', 'Ano IGC']
colunas_emec_cursos = ['Código IES', 'Código Curso', 'CC', 'Ano CC', 'CPC', 'Ano CPC', 'ENADE', 'Ano ENADE', 'IDD', 'Ano IDD', 'Situação', 'Vagas Autorizadas']

### LEGENDA DOS DADOS

## Dados da IES
# NU_ANO_CENSO: Ano do Censo da IES (Fonte: Microdados IES)
# SG_UF_IES: Sigla da Unidade Federativa da IES
# NO_MUNICIPIO_IES: Nome do município da IES
# TP_ORGANIZACAO_ACADEMICA: Tipo de organização acadêmica da IES
# TP_REDE: Tipo de rede da IES (pública ou privada)
# CO_IES: Código da IES
# NO_IES: Nome da IES

## Dados do curso
# SG_UF: Sigla da Unidade Federativa do Curso
# NO_MUNICIPIO: Nome do município do Curso
# NO_CURSO: Nome do curso
# CO_CURSO: Código do curso
# NO_CINE_ROTULO: Nome do CINE (Classificação Nacional de Educação Superior) - Rótulo
# NO_CINE_AREA_ESPECIFICA: Nome da área específica do CINE
# TP_GRAU_ACADEMICO: Tipo de grau acadêmico do curso
# IN_GRATUITO: Indicador de gratuidade do curso
# TP_MODALIDADE_ENSINO: Tipo de modalidade de ensino do curso
# QT_VG_TOTAL: Quantidade de vagas totais do curso
# QT_INSCRITO_TOTAL: Quantidade de inscritos totais no curso

## Dados E-MEC IES
# CI: Conceito Institucional
# CI-EaD: Conceito Institucional do Ensino a Distância
# IGC: Índice Geral de Cursos
# Ano CI, Ano CI-EaD, Ano IGC: Ano em que foi realizada a avaliação

## Dados E-MEC Curso
# CC: Conceito de Curso - nota de 1 a 5 atribuída após avaliação presencial do INEP/MEC
# Ano CC: Ano em que foi realizada a avaliação presencial que gerou o CC
# CPC: Conceito Preliminar de Curso - indicador de 1 a 5 que avalia qualidade dos cursos de graduação
# Ano CPC: Ano em que foi calculado o Conceito Preliminar de Curso
# ENADE: Nota de 1 a 5 do Exame Nacional de Desempenho dos Estudantes
# Ano ENADE: Ano em que o curso participou do ENADE
# IDD: Indicador de Diferença entre Desempenhos Observado e Esperado - mede valor agregado pelo curso
# Ano IDD: Ano em que foi calculado o indicador IDD
# Situação: Status atual do curso (Ativo, Suspenso, Extinto, etc.)
# Vagas Autorizadas: Número de vagas que o MEC autorizou para o curso

# Carregar os dados das bases, especificando as colunas a serem lidas
df_ies = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/MICRODADOS_ED_SUP_IES_2023.CSV', 'csv', colunas_ies, sep=';', encoding='latin-1', low_memory=False)
if df_ies is None:
    exit()

df_cursos = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/MICRODADOS_CADASTRO_CURSOS_2023.CSV', 'csv', colunas_cursos, sep=';', encoding='latin-1', low_memory=False)
if df_cursos is None:
    exit()

df_emec_ies = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/consulta-publica-emec-ies-20241205.xlsx', 'excel', colunas_emec_ies)
if df_emec_ies is None:
    exit()

df_emec_cursos = load_dataframe('/content/drive/MyDrive/Projeto Ext Faculdade Exame/consulta-publica-emec-cursos-20241205.xlsx', 'excel', colunas_emec_cursos)
if df_emec_cursos is None:
    exit()

In [18]:
# Renomear colunas
df_emec_ies = df_emec_ies.rename(columns={'Código IES': 'CO_IES'})
df_emec_cursos.rename(columns={'Código Curso': 'CO_CURSO'}, inplace=True)
df_emec_cursos.drop_duplicates(subset=['CO_CURSO'], inplace=True)


# Juntar os dataframes
df_ies_e_cursos = pd.merge(df_ies, df_cursos, on='CO_IES', how='inner')
df_merged = pd.merge(df_ies_e_cursos, df_emec_ies, on='CO_IES', how='inner')
df_merged = pd.merge(df_merged, df_emec_cursos, on='CO_CURSO', how='left')

# Remover colunas duplicadas (_x e _y)
colunas_originais = {}
for col in df_merged.columns:
    if '_x' in col or '_y' in col:
        col_original = col.replace('_x', '').replace('_y', '')
        if col_original in colunas_originais:
            # Se a coluna original já foi encontrada, remova a coluna atual
            df_merged.drop(columns=[col], inplace=True)
        else:
            # Se a coluna original ainda não foi encontrada, renomeie a coluna atual
            colunas_originais[col_original] = col
            df_merged.rename(columns={col: col_original}, inplace=True)

# Exportar para CSV
try:
    df_merged.to_csv('dados_combinados.csv', index=False, encoding='utf-8')
    print("Arquivo dados_combinados.csv criado com sucesso!")
except Exception as e:
    print(f"Erro ao exportar o arquivo: {e}")

# Exibir informações e realizar verificações
#print(df_merged.info())
#print(df_merged.describe())
#print(df_merged.head())

#print(f"O DataFrame df_merged possui {df_merged.shape[0]} linhas e {df_merged.shape[1]} colunas.")

# Filtrar e exibir cursos para um CO_IES específico
#cursos_co_ies = df_merged[df_merged['CO_IES'] == 322]
#print(cursos_co_ies)

#Imprimir colunas para verificação
#print("Colunas df_cursos:", df_cursos.columns.tolist())
#print("\nColunas df_ies:", df_ies.columns.tolist())
#print("\nColunas df_emec_ies:", df_emec_ies.columns.tolist())
#print("\nColunas df_emec_cursos:", df_emec_cursos.columns.tolist())
#print("\nColunas df_merged:", df_merged.columns.tolist())

Arquivo dados_combinados.csv criado com sucesso!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671507 entries, 0 to 671506
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NU_ANO_CENSO              671507 non-null  int64  
 1   SG_UF_IES                 671507 non-null  object 
 2   NO_MUNICIPIO_IES          671507 non-null  object 
 3   TP_ORGANIZACAO_ACADEMICA  671507 non-null  int64  
 4   TP_REDE                   671507 non-null  int64  
 5   CO_IES                    671507 non-null  int64  
 6   NO_IES                    671507 non-null  object 
 7   SG_UF                     660476 non-null  object 
 8   NO_MUNICIPIO              660476 non-null  object 
 9   NO_CURSO                  671507 non-null  object 
 10  CO_CURSO                  671507 non-null  int64  
 11  NO_CINE_ROTULO            671507 non-null  object 
 12  NO_CINE_AREA_ESPECIFICA   671507 non-null  object 


In [33]:
import os

# Código que será escrito no arquivo app.py
codigo = '''import streamlit as st
import pandas as pd
from st_aggrid import AgGrid, GridOptionsBuilder
from st_aggrid.shared import GridUpdateMode

# Configuração da página
st.set_page_config(layout="wide", page_title="Análise de Dados do MEC")

# Carregar os dados
@st.cache_data
def load_data():
    df = pd.read_csv('dados_combinados.csv', encoding='utf-8', low_memory=False)
    # Converter colunas para tipos numéricos, tratando valores não numéricos
    for col in ['CI', 'CI-EaD', 'IGC', 'CC', 'CPC', 'ENADE', 'IDD']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

df_merged = load_data()

# Sidebar com filtros
st.sidebar.title("Filtros")

# Filtros para IES
uf_filter = st.sidebar.multiselect('UF', df_merged['SG_UF_IES'].unique())
org_academica_filter = st.sidebar.multiselect('Organização Acadêmica', df_merged['NO_IES'].unique())
rede_filter = st.sidebar.multiselect('Tipo de Rede', ['Pública', 'Privada'])

# Filtros para Cursos
situacao_filter = st.sidebar.selectbox('Situação do Curso', ['Todos'] + list(df_merged['Situação'].unique()))

# Função para aplicar filtros
def apply_filters(df):
    if uf_filter:
        df = df[df['SG_UF_IES'].isin(uf_filter)]
    if org_academica_filter:
        df = df[df['NO_IES'].isin(org_academica_filter)]
    if rede_filter:
        df = df[df['TP_REDE'].isin([1 if 'Pública' in rede_filter else 2 if 'Privada' in rede_filter else None])]
    if situacao_filter != 'Todos':
        df = df[df['Situação'] == situacao_filter]
    return df

# Aplicar filtros
filtered_df = apply_filters(df_merged)

# Função para criar tabela com AgGrid
def create_aggrid(df, columns, key):
    gb = GridOptionsBuilder.from_dataframe(df[columns])
    gb.configure_pagination(paginationAutoPageSize=True)
    gb.configure_side_bar()
    gb.configure_selection('single', use_checkbox=True, groupSelectsChildren="Group checkbox select children")
    gb.configure_default_column(sorteable=True, filterable=True)
    gridOptions = gb.build()

    return AgGrid(
        df[columns],
        gridOptions=gridOptions,
        data_return_mode='AS_INPUT',
        update_mode=GridUpdateMode.MODEL_CHANGED,
        fit_columns_on_grid_load=False,
        theme='streamlit',
        enable_enterprise_modules=True,
        height=400,
        width='100%',
        reload_data=True,
        key=key
    )

# Visualização 1: Ranking IES
st.title("Visualização 1: Ranking IES")

ies_columns = ['CO_IES', 'NO_IES', 'SG_UF_IES', 'NO_MUNICIPIO_IES', 'TP_ORGANIZACAO_ACADEMICA', 'TP_REDE',
               'CI', 'CI-EaD', 'IGC', 'Ano CI', 'Ano CI-EaD', 'Ano IGC']

ies_df = filtered_df.drop_duplicates(subset=['CO_IES'])[ies_columns]
ies_df['TP_REDE'] = ies_df['TP_REDE'].map({1: 'Pública', 2: 'Privada'})

create_aggrid(ies_df, ies_columns, 'ies_table')

# Visualização 2: Ranking Cursos
st.title("Visualização 2: Ranking Cursos")

cursos_columns = ['CO_IES', 'NO_IES', 'NO_CURSO', 'CC', 'Ano CC', 'CPC', 'ENADE', 'Ano ENADE', 'IDD', 'Situação']

cursos_df = filtered_df[cursos_columns]

create_aggrid(cursos_df, cursos_columns, 'cursos_table')
'''

# Criando o arquivo app2.py e escrevendo o código nele
with open('app.py', 'w') as file:
    file.write(codigo)

print("Arquivo app.py criado com sucesso!")

Arquivo app3.py criado com sucesso!


In [19]:
# Instalando Streamlit para gerar uma visualização web
!pip install -q streamlit
!pip install streamlit-aggrid
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 25 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [35]:
# Imprimir o IP para usar no localtunnel
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run app.py & npx localtunnel --port 8501

Password/Endpoint IP for localtunnel is: 34.91.119.47


⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.119.47:8501

your url is: https://nasty-ears-work.loca.lt
2024-12-07 02:56:03.475 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='nasty-ears-work.loca.lt', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tornado/websocket.py", line 937, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/server/browser_websocket_handler.py", line 126, in open
    self._session_id = self._runtime.connect_session(
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/runtime.py", line 384, in connect_session
    session